# Univariate Fine-Mapping and TWAS with SuSiE


## Description

Our pipeline is capable of performing univariate fine-mapping with SuSiE with TWAS weights. The TWAS portion makes use of TWAS weights, linkage disequilibrium data and GWAS summary statistics. Preset variants used are taken from the linkage disequilibrium data and used only for TWAS. TWAS cross validation tell us which of the four methods (enet, lasso, mrash, SuSiE) are best to use. By default, we limit to under 5000 variants for cross validation. In cross validation, the data is split into five parts. Training is done on four parts, and prediction is done on the fifth. Linear regression is used to assess the results and get r squared and pvalues. 

Fine mapping with SuSiE follows the formulat y=xb+e where x has many highly correlated variables due to linkage disequilibrium. Therefore, true effects (b), are very sparse. The SuSiE wrapper looks for five independent signals in each region to increase convergence speed. However, if five signals are found, then the the upper limit is increased. SuSiE does not allow for the inclusion of covariates. Therefore, covariates are regressed in.

## Input:

    
`--genoFile`: path to a plink bed file containing genotypes.  
`--phenoFile`: at least one tab delimited file containing chr, start, end, ID and path for the regions. For example:
```
#chr    start   end     ID      path
chr12   389272  389273  ENSG00000120647 $PATH/Ast.log2cpm.bed.gz
chr12   389319  389320  ENSG00000073614 $PATH/Ast.log2cpm.bed.gz
```

`--covFile`: at least one tab delimited file containing covariates in the rows and samples in the columns.  
`--customized-association-windows`: a tab delimited file containing chr, start, end, and gene_id. For example:
```
#chr    start   end     gene_id
chr1    0       6480000 ENSG00000008128
chr1    0       6480000 ENSG00000008130
```
`----phenotype-names`: the names of the phenotypes if multiple are included. There should be one for each phenotype file you include.   
`--max-cv-variants`: maximum number of variants for cross-validation.  
`--ld_reference_meta_file`: path to file containing chrom, start, end and path for linkage disequilibrium region information. For example:
```
#chrom  start   end     path
chr1    101384274       104443097       $PATH/chr1_101384274_104443097.cor.xz.bim
chr1    104443097       106225286       $PATH/chr1_104443097_106225286.cor.xz.bim
```
`--region-name`: if you only wish to analyze one region, then include the gene_id of a region found in the `customized-association-windows` file

## Minimal Working Example Steps

### i. Run the Fine-Mapping and TWAS with SuSiE

In [ ]:
sos run pipeline/mnm_regression.ipynb susie_twas \
    --name test_susie_twas \
    --genoFile output/genotype_by_chrom/wgs.merged.plink_qc.1.bed \
    --phenoFile output/phenotype/phenotype_by_chrom_for_cis/bulk_rnaseq.phenotype_by_chrom_files.region_list.txt \
    --covFile output/covariate/bulk_rnaseq_tmp_matrix.low_expression_filtered.outlier_removed.tmm.expression.covariates.wgs.merged.plink_qc.plink_qc.prune.pca.Marchenko_PC.gz \
    --customized-association-windows reference_data/TAD/TADB_enhanced_cis.bed \
    --phenotype-names test_pheno \
    --max-cv-variants 5000 --ld_reference_meta_file data/ld_meta_file_with_bim.tsv \
    --region-name ENSG00000049246 ENSG00000054116 ENSG00000116678 \
    --save-data \
    --cwd output/mnm_regression/susie_twas

```
INFO: Running get_analysis_regions: 
Loading customized association analysis window from reference_data/TAD/TADB_enhanced_cis.bed
INFO: get_analysis_regions is completed.
INFO: get_analysis_regions output:   regional_data
INFO: Running susie_twas: 
INFO: susie_twas (index=0) is completed.
INFO: susie_twas (index=1) is completed.
INFO: susie_twas (index=2) is completed.
INFO: susie_twas output:   /restricted/projectnb/xqtl/xqtl_protocol/toy_xqtl_protocol/output/mnm_regression/susie_twas/fine_mapping/test_susie_twas.chr1_ENSG00000049246.univariate_bvsr.rds /restricted/projectnb/xqtl/xqtl_protocol/toy_xqtl_protocol/output/mnm_regression/susie_twas/twas_weights/test_susie_twas.chr1_ENSG00000049246.univariate_twas_weights.rds... (6 items in 3 groups)
INFO: Workflow susie_twas (ID=w487c342397d09ccb) is executed successfully with 2 completed steps and 4 completed substeps.
```

## Anticipated Results

Univariate finemapping will produce a file containing results for the top hits and a file containing twas weights produced by susie. 

`ROSMAP_mega_eQTL.chr11_ENSG00000073921.univariate_bvrs.rds`:
* For each gene of interest and phenotype, this file contains:
    1. susie_fitted
    2. variant_names
    3. analysis_script
    4. other quantities - information on dropped samples, if any
    5. summary statistics (beta and se) for each variant
    6. sample names
    7. summary statistics for top loci
    8. susie results trimmed - includes pip, sets, cs_corr, sets_secondary, alpha, lbf_variable, V, niter, X_column_scale_factors, mu, mu2
    9. total time elapsed
    10. region info
    11. preset_variants_results

`ROSMAP_mega_eQTL.chr11_ENSG00000073921.univariate_data.rds` (from the --save-data argument):
* For each gene of interest, contains residuals for each sample and phenotype
* see [pecotmr code](https://github.com/statfungen/pecotmr/blob/68d87ca1d0a059022bf4e55339621cbddc8993cc/R/file_utils.R#L461) for description at `load_regional_association_data` function

`ROSMAP_mega_eQTL.chr11_ENSG00000073921.univariate_twas_weights.rds`:
* For each gene of interest and phenotype, this file contains:
    1. twas_weights - weights for enet, lasso, bayes r, mrash and susie methods
    2. twas_predictions - twas predictions for enet, lasso, bayes r, mrash and susie methods
    3. twas_cv_result
    4. total_time_elapsed
    5. variant_names
    6. region_info
